In [1]:
from scipy import stats as st
import pandas as pd
import math
import numpy as np

In [2]:
des_df = pd.read_excel('data/LibraryDesign.xls')
nat_df = pd.read_excel('data/LibraryNatural.xls')

In [5]:
globalmmd_df = des_df[des_df.header.str.contains('mmd_')]

# sort the values in terms of norm r.e.
globalmmd_df = globalmmd_df.sort_values('RE_norm')
nat_df = nat_df.sort_values('RE_norm')

# drop any empty values in terms of norm r.e.:
globalmmd_df = globalmmd_df.dropna(subset = ['RE_norm'])
nat_df = nat_df.dropna(subset = ['RE_norm'])

In [7]:
num_globalmmd, num_nat = globalmmd_df.shape[0], nat_df.shape[0]

In [8]:
""" The null hypothesis for 2-sample test is rejected for large samples from N and M distributions. In particular,

D < c(alpha) sqrt( (n+m)/(n*m) ), where c(alpha) and alpha is found from a lookup table and is choosen depending on 
the degree of stringency. For example, higher alpha value means easier to not reject the null hypothesis, while lower
alpha means it is easier to reject the null hypothesis. Usually, alpha = 0.05 is choosen, so c(alpha) is 1.358 (ref: https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test).
"""

def check_nullH_2smaple(D, alpha, n, m):
    """
    function:
    
    arg:
    
    return:
    
    """
    c = {0.20:1.073, 0.15:1.138, 0.10:1.224, 0.05:1.358, 0.025:1.48, 0.01:1.628, 0.005:1.731, 0.001:1.949}
    c_alpha = c[alpha]
    null_threshold = c_alpha * math.sqrt( (n+m)/ (n*m))
    if D < null_threshold:
        print('Null hypothesis cannot be rejected')
        decision = 'Keep'
    else:
        print('Reject null hypothesis')
        decision = 'Reject'
    
    return decision

## KS 2 smaple test for global design and natural homolog sequences:

In [9]:
# norm re scores
global_normre = globalmmd_df.RE_norm.values
nat_normre = nat_df.RE_norm.values

# compute ks test for 2 sample 
twosample_re = st.ks_2samp(global_normre, nat_normre, 'two-sided')


# ks statistics and p value
D_2test = twosample_re[0]
p_2test = twosample_re[1]

# compute if the null hypothesis is rejected or not. 
alpha = 0.05
decision_2smaple = check_nullH_2smaple(D_2test, alpha, len(global_normre), len(nat_normre))

Reject null hypothesis


In [11]:
# compute 1sample test and compare if the global design sequences are normally distribution 
onesample_global = st.kstest(global_normre, 'norm')

# ks statistics and p value
D_1test_globalmmd = onesample_global[0]
p_1test_globalmmd = onesample_global[1]

print(f'Since the p value {p_1test_globalmmd} < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.')

Since the p value 1.524786301685347e-219 < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.


In [12]:
# compute 1sample test and compare if the natural sequences are normally distribution 
onesample_nat = st.kstest(nat_normre, 'norm')

# ks statistics and p value
D_1test_nat = onesample_nat[0]
p_1test_nat = onesample_nat[1]

print(f'Since the p value {p_1test_nat} < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.')

Since the p value 0.0 < 0.05, the null hypothesis can be rejected. Therefore, the distribution is not normally distributed.
